In [12]:
from pyspark.sql import SparkSession, Row, Window
from pyspark.sql.functions import *
from pyspark import SparkConf
from pyspark.sql.types import IntegerType


sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("GCSExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set(
    "fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS"
)

# Use the Cloud Storage bucket for temporary BigQuery export data the connector uses.
bucket = "temp_de2023_20204025"
spark.conf.set("temporaryGcsBucket", bucket)

#  Google Storage File Path
gsc_file_path = "gs://data_de2023_20204025"  #  use your gcp bucket name.
# Create data frame
movie_df = (
    spark.read.format("csv")
    .option("header", "true")
    .load(gsc_file_path + "/rotten_tomatoes_movies.csv")
)
movie_df.printSchema()

movie_df.show()

root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- audienceScore: string (nullable = true)
 |-- tomatoMeter: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- ratingContents: string (nullable = true)
 |-- releaseDateTheaters: string (nullable = true)
 |-- releaseDateStreaming: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- originalLanguage: string (nullable = true)
 |-- director: string (nullable = true)
 |-- writer: string (nullable = true)
 |-- boxOffice: string (nullable = true)
 |-- distributor: string (nullable = true)
 |-- soundMix: string (nullable = true)

+--------------------+--------------------+-------------+-----------+------+--------------------+-------------------+--------------------+--------------+--------------------+----------------+--------------------+--------------------+---------+-----------------+--------+
|                  id|               title|a

In [13]:
# Handling missing values
movie_df = (
    movie_df.withColumn(
        "age_class", when(col("rating").isNotNull(), col("rating")).otherwise("G")
    )
    .withColumn(
        "ratingContents",
        when(col("ratingContents").isNotNull(), col("ratingContents")).otherwise(
            "None"
        ),
    )
    .withColumn(
        "distributor",
        when(col("distributor").isNotNull(), col("distributor")).otherwise(
            "independent"
        ),
    )
)

# Dropping columns
columns_to_drop = ["soundMix", "boxOffice", "rating"]
movie_df = movie_df.drop(*columns_to_drop)

# audienceScore to integer
movie_df = movie_df.withColumn(
    "audienceScore", movie_df["audienceScore"].cast(IntegerType())
)

# Dropping rows with missing values in specific columns
columns_to_check_for_null = [
    "audienceScore",
    "tomatoMeter",
    "releaseDateTheaters",
    "runtimeMinutes",
    "genre",
    "director",
    "originalLanguage",
]
movie_df = movie_df.dropna(subset=columns_to_check_for_null)

# Converting releaseDateTheaters to datetime
movie_df = movie_df.withColumn(
    "releaseDateTheaters", to_date(col("releaseDateTheaters"), "y-M-d")
)  # our dataframe with date column

# Extracting date components
movie_df = (
    movie_df.withColumn("releaseDay", day(col("releaseDateTheaters")))
    .withColumn("releaseMonth", month(col("releaseDateTheaters")))
    .withColumn("releaseYear", year(col("releaseDateTheaters")))
    .withColumn("releaseWeekday", weekday(col("releaseDateTheaters")))
)

# Show the resulting PySpark DataFrame
movie_df.show()

+--------------------+--------------------+-------------+-----------+--------------------+-------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+------------+-----------+--------------+
|                  id|               title|audienceScore|tomatoMeter|      ratingContents|releaseDateTheaters|releaseDateStreaming|runtimeMinutes|               genre|    originalLanguage|            director|              writer|         distributor|age_class|releaseDay|releaseMonth|releaseYear|releaseWeekday|
+--------------------+--------------------+-------------+-----------+--------------------+-------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+---------+----------+------------+-----------+--------------+
|         adrift_2018|              Adrift|           65|    

In [14]:
# Rank the resturants in terms of the avg price, per each city. Then, select the best and worst product using ranks
windowasc = Window.partitionBy(col("genre"), col("releaseYear")).orderBy(
    col("audienceScore").asc()
)

movie_df_window = movie_df.withColumn("rank", dense_rank().over(windowasc))

movie_df_window.select(
    col('id'), col("title"), col("audienceScore"), col("genre"), col("releaseYear"), col("rank")
).write.format("bigquery").option(
    "table", "de-23-lab-1-399021.assignment2dataset.movierank"
).mode(
    "overwrite"
).save()

movie_df_window.select(
    col("title"), col("audienceScore"), col("genre"), col("releaseYear"), col("rank")
).write.format("bigquery").option(
    "table", "de-23-lab-1-399021.assignment2dataset.movierank_without_id"
).mode(
    "overwrite"
).save()

movie_df_window.select(
    col("title"), col("audienceScore"), col("genre"), col("releaseYear"), col("rank")
).show(100)

+--------------------+-------------+------+-----------+----+
|               title|audienceScore| genre|releaseYear|rank|
+--------------------+-------------+------+-----------+----+
|The Charge of the...|           60|Action|       1936|   1|
|       Gentleman Jim|           85|Action|       1942|   1|
|   Sign of the Pagan|           33|Action|       1954|   1|
|Riot in Cell Bloc...|           77|Action|       1954|   2|
|           Moonfleet|           71|Action|       1955|   1|
|       Seven Samurai|           97|Action|       1956|   1|
|The Pride and the...|           25|Action|       1957|   1|
|       A Man Escaped|           93|Action|       1957|   2|
|  The Horse Soldiers|           75|Action|       1959|   1|
|From Russia With ...|           84|Action|       1963|   1|
|          Goldfinger|           89|Action|       1964|   1|
|         Thunderball|           73|Action|       1965|   1|
|     The Wild Angels|           46|Action|       1966|   1|
|              Batman|  

In [15]:
movie_df.groupby("age_class").agg(
    avg("audienceScore").alias("avg_audience_score")
).write.format("bigquery").option(
    "table", "de-23-lab-1-399021.assignment2dataset.ageperformance"
).mode(
    "overwrite"
).save()

movie_df.groupby("age_class").agg(
    avg("audienceScore").alias("avg_audience_score")
).show()

+---------+------------------+
|age_class|avg_audience_score|
+---------+------------------+
|     TV14|              30.0|
|       PG| 64.90593047034764|
|     TVPG|              81.0|
|    NC-17|56.642857142857146|
|     TVY7|              68.0|
|        R|57.960393772893774|
|        G|  65.6764733092793|
|    PG-13| 61.66748366013072|
|     TVMA|              63.5|
+---------+------------------+



In [16]:
spark.stop()